In [1]:
import pandas as pd
import numpy as np

from m4_feats_polars import *
from m5_sb_models import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
import polars as pl
data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

In [3]:
# everything is logged
# bursts = 2/3 of a second - input only
# inter word pauses
# between sentence pauses ?
# between paragraph pauses ?
# backspace pauses
# edit pauses

In [4]:
Best_Feature_Set = [
    'train_down_events_counts.pkl', 
    'train_vector_one_gram.pkl', 
    'train_create_pauses.pkl', 
    'train_essay_paragraphs.pkl', 
    'train_cursor_pos_acceleration.pkl', 
    'train_word_count_acceleration.pkl', 
    'train_vector_two_gram.pkl']

# (action_time_by_activity, categorical_nunique) # best 2 0.602997
('get_keys_pressed_per_second', 'p_burst_feats', 'r_burst_feats') # best 3 0.602696

best_feature_set_2 = [
    'train_down_events_counts.pkl',
    'train_vector_one_gram.pkl',
    'train_create_pauses.pkl',
    'train_essay_paragraphs.pkl',
    'train_essay_sentences.pkl',
    'train_cursor_pos_acceleration.pkl',
    'train_word_count_acceleration.pkl',
    'train_categorical_nunique.pkl',
    'train_cursor_pos_rate_of_change.pkl',
    'train_get_keys_pressed_per_second.pkl']   

base_feat_3 = ['train_down_events_counts.pkl', 
               'train_vector_one_gram.pkl', 
               'train_create_pauses.pkl', 
               'train_essay_paragraphs.pkl', 
               'train_cursor_pos_acceleration.pkl', 
               'train_essay_sentences.pkl', 
               'train_action_time_baseline_stats.pkl']

In [5]:
# best_feature_set_1 - PARTIAL

train_essays          = get_essay_df(train_logs.collect().to_pandas())
test_essays           = get_essay_df(test_logs.collect().to_pandas())

# train_down_events_counts
tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
# train_vector_one_gram
tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
# train_create_pauses
tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
# train_cursor_pos_acceleration
tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
# train_word_count_acceleration
tr_word_count_acc, ts_word_count_acc = word_count_acceleration(train_logs, test_logs)
# train_vector_two_gram
tr_vect_two, ts_vect_two = countvectorize_two_one(train_essays, test_essays)
# time by activity
tr_time_by_act, ts_time_by_act = action_time_by_activity(train_logs, test_logs)

# count_of_activities(train_logs, test_logs)
tr_act_count, ts_act_count = count_of_activities(train_logs, test_logs)
# # r_burst_feats
tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
# # p_burst_feats
tr_p_burst, ts_p_burst = p_burst_feats(train_logs, test_logs, 2)
# nunique
tr_nunique, ts_nunique = categorical_nunique(train_logs, test_logs)
# get_keys_pressed_per_second
tr_get_keys, ts_get_keys = get_keys_pressed_per_second(train_logs.collect().to_pandas(), 
                                                       test_logs.collect().to_pandas())

# input change feats
tr_input_change, ts_input_change = input_text_change_feats(train_logs, test_logs)
#  word_counts_rate_of_change

tr_wc_roc, ts_wc_roc =  word_counts_rate_of_change(train_logs, test_logs)

['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 'train_create_pauses.pkl', 'train_essay_paragraphs.pkl', 'train_cursor_pos_acceleration.pkl']

# (action_time_by_activity, categorical_nunique) # best 2 0.602997
# ('get_keys_pressed_per_second', 'p_burst_feats', 'r_burst_feats') # best 3 0.602696

train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
train_feats = train_feats.join(tr_pauses, on='id', how='left')
train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
# train_feats = train_feats.join(tr_word_count_acc, on='id', how='left')
# train_feats = train_feats.join(tr_vect_two, on='id', how='left')
# train_feats = train_feats.join(tr_r_burst, on='id', how='left')
# train_feats = train_feats.join(tr_get_keys, on='id', how='left')
# train_feats = train_feats.join(tr_wc_roc, on='id', how='left')
# train_feats = train_feats.join(tr_input_change, on='id', how='left')
# train_feats = train_feats.join(tr_nunique, on='id', how='left')
train_feats = train_feats.join(tr_p_burst, on='id', how='left')
# train_feats = train_feats.join(tr_time_by_act, on='id', how='left')


test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
test_feats = test_feats.join(ts_pauses, on='id', how='left')
test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
# test_feats = test_feats.join(ts_word_count_acc, on='id', how='left')
# test_feats = test_feats.join(ts_vect_two, on='id', how='left')
# test_feats = test_feats.join(ts_r_burst, on='id', how='left')
# test_feats = test_feats.join(ts_get_keys, on='id', how='left')
# test_feats = test_feats.join(ts_wc_roc, on='id', how='left')
# test_feats = test_feats.join(ts_input_change, on='id', how='left')
# test_feats = test_feats.join(ts_nunique, on='id', how='left')
test_feats = test_feats.join(ts_p_burst, on='id', how='left')
# test_feats = test_feats.join(ts_time_by_act, on='id', how='left')


train_logs = train_logs.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
train_scores = train_scores.collect().to_pandas()
train_feats = train_feats.sort('id')
train_feats = train_feats.collect().to_pandas()
test_feats = test_feats.collect().to_pandas()

train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')

# train_feats           = train_feats.merge(sent_feats(train_essays), on='id', how='left')
# test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')


train_feats           = train_feats.merge(train_scores, on=['id'], how='left')
train_feats           = train_feats.sort_values('id')

print(f'train feats shape {train_feats.shape}')

< Events counts features >
< Count vectorize one-grams >
< Idle time features >
< cursor position acceleration >
< word count acceleration >
< Count vectorize bi-grams >
< Action time by activities >
< R-burst features >
< P-burst features >
< Categorical # unique values features >
< Input text change features >
< Word counts rate of change features >
< Essays paragraphs feats >
< Essays paragraphs feats >
train feats shape (2471, 86)


In [6]:
# missing_cols = set(train_feats.columns) - set(test_feats.columns)
# missing_cols.remove('score')

# for col in missing_cols:
#     test_feats[col] = np.nan

# for estimators in [350]:

lgb_params_1 = {
    'boosting_type': 'gbdt', 
    'metric': 'rmse',
    'reg_alpha': 0.0031, 
    'reg_lambda': 0.001, 
    'colsample_bytree': 0.8,  
    'subsample_freq': 1,  
    'subsample': 0.75,  
    'learning_rate': 0.017, 
    'num_leaves': 19, 
    'min_child_samples': 46,
    'n_estimators': 350,
    'verbosity': -1
    }

param = {'n_estimators': 1024,
        'learning_rate': 0.005,
        'metric': 'rmse',
        'force_col_wise': True,
        'verbosity': 0,}
    
#print(f'Number of estimators: {estimators}')
test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, lgb_params_1)
#test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, param)

Final RMSE over 50: 0.605257. Std 0.8248
RMSE by fold 0.605152. Std 0.0113


new_p_burst 2/3
Final RMSE over 50: 0.605179. Std 0.8250 
RMSE by fold 0.605076. Std 0.0113
Final RMSE over 50: 0.605200. Std 0.8245
RMSE by fold 0.605084. Std 0.0120
Final RMSE over 50: 0.605611. Std 0.8250
RMSE by fold 0.605490. Std 0.0122
Final RMSE over 50: 0.605304. Std 0.8246
RMSE by fold 0.605189. Std 0.0119

-------------------------------------------
new_p_burst 2
Final RMSE over 50: 0.604866. Std 0.8248
RMSE by fold 0.604760. Std 0.0114
Final RMSE over 50: 0.605266. Std 0.8250
RMSE by fold 0.605167. Std 0.0110
Final RMSE over 50: 0.605132. Std 0.8249
RMSE by fold 0.605019. Std 0.0118
Final RMSE over 50: 0.605257. Std 0.8248
RMSE by fold 0.605152. Std 0.0113


reduced feats
Final RMSE over 50: 0.604472. Std 0.8245
RMSE by fold 0.604365. Std 0.0115

r burst only
Number of estimators: 350
Final RMSE over 50: 0.604057. Std 0.8244
RMSE by fold 0.603946. Std 0.0116

Number of estimators: 350
Final RMSE over 50: 0.604650. Std 0.8243
RMSE by fold 0.604527. Std 0.0122

In [7]:
import lightgbm as lgb

# Assuming X_train and y_train are your features and labels respectively
weights = [1.0 for _ in range(len(y_train))]  # Default weights
weights[some_specific_index] = 1.5  # Higher weight for a specific instance

train_data = lgb.Dataset(X_train, label=y_train, weight=weights)
# Proceed with setting up parameters and training the model


NameError: name 'y_train' is not defined

In [ ]:
test_ids = test_feats.id
y_pred = np.mean(test_preds, axis=0)

sub = pd.DataFrame({'id': test_ids, 'score': y_pred})
sub.to_csv('submission.csv', index=False)